In [1]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import librosa
import librosa.display
import IPython.display as ipd

from glob import glob
import tensorflow as tf

#to make plots look nice
from itertools import cycle
sns.set_theme(style="white", palette=None)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import cross_validate

2022-11-23 15:12:55.619323: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-23 15:12:55.619389: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Importing audio files

In [2]:
audio_files = glob("data/Emotion_Dataset/*/*.wav")

# Preprocessing

- vectorizing every files with librosa load
- trimming the blank spaces
- padding (is it relevant with random forest?)
- melspectrogram
- appending to a list
- making it a numpy array

In [3]:
spectrograms = []
for item in audio_files:
    X, sr = librosa.load(item) #vectorizing
    X_trim = librosa.effects.trim(X,top_db=35) #trimming data
    
    #padding
    if X_trim[0].shape[0] >= 65000:
        X_final = X_trim[0][:65000]
        X_final = tf.convert_to_tensor(X_final).numpy()
    else:
        zero_padding = tf.zeros([65000]-tf.shape(X_trim[0]),dtype=tf.float32)
        X_final = tf.concat([X_trim[0],zero_padding],0).numpy()
        
    #mel spectogram
    S = librosa.feature.melspectrogram(y=X_final,
                                   sr=sr,
                                   n_mels=128,)
    
    #S = tf.expand_dims(S, axis=2)
    spectrograms.append(S)

2022-11-23 15:13:00.932950: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-23 15:13:00.935177: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-23 15:13:00.935430: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-23E8V41): /proc/driver/nvidia/version does not exist
2022-11-23 15:13:00.955583: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
X= np.array(spectrograms)

In [14]:
X.shape

(1440, 128, 127)

# Target Encoding

In [44]:
ids=[]
for audio in audio_files:
    ids.append(audio[30:-4])

new_df = pd.DataFrame(ids)
new_df.rename(columns={0: "titulo"},inplace=True)
new_df['emotion'] = new_df.apply(lambda x: x[0].split("-")[2], axis=1)
y = new_df['emotion']

In [51]:
pd.DataFrame(y)

,emotion
0,01
1,06
2,08
3,05
4,06
...,...
1435,02
1436,06
1437,04
1438,08


# Splitting train test

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modeling Random Forest

## flattening the spectogram

In [47]:
X_flat=X_train.reshape(1008,128*127)

In [54]:
y_train.shape

(1008,)

In [48]:
X_flat.shape

(1008, 16256)

## random forest

In [90]:
clf = RandomForestClassifier(max_depth=10,n_estimators=1000)
clf.fit(X_flat,y_train)

RandomForestClassifier(max_depth=10, n_estimators=1000)

## Prediction and metrics

In [91]:
y_pred=clf.predict(X_test.reshape(432,128*127))
#cross_validate(clf,y_test,y_pred,scoring='accuracy')

In [92]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5370370370370371

In [108]:
df1 = pd.DataFrame(y_test).reset_index()
df2 = pd.DataFrame(y_pred)

In [107]:
df2

,0
0,03
1,07
2,07
3,05
4,06
...,...
427,08
428,08
429,03
430,07


In [115]:
df_check = df1.join(df2)
df_check.rename(columns={0:'pred'})

,index,emotion,pred
0,168,03,03
1,605,07,07
2,548,04,07
3,65,03,05
4,628,06,06
...,...,...,...
427,591,05,08
428,1133,05,08
429,1190,03,03
430,438,07,07
